In [1]:
import numpy as np
import scipy.stats as stats
from tqdm import tqdm

In [2]:
def brute_force_coincidence_fraction(lambda_G, lambda_S, delta_t, num_simulations=500):

    total_events = 0
    total_coincident_events = 0

    for _ in range(num_simulations):
        n_G = np.random.poisson(lambda_G)
        n_S = np.random.poisson(lambda_S)

        if n_G == 0 and n_S == 0:
            continue

        times_G = np.random.uniform(0, 1, n_G)
        times_S = np.random.uniform(0, 1, n_S)

        if n_G > 0 and n_S > 0:
            dt = np.abs(times_G[:, None] - times_S[None, :])

            dt = times_S[None, :] - times_G[:, None]

            G_has_match = np.any((dt > 0) & (dt <= delta_t), axis=1)
            S_has_match = np.any((dt > 0) & (dt <= delta_t), axis=0)

            coincident = G_has_match.sum() + S_has_match.sum()
        else:
            coincident = 0

        total_coincident_events += coincident
        total_events += n_G + n_S
        # break

    return total_coincident_events / total_events


In [3]:

lambda_G = 4264/14# rate of signal G (events per year)
lambda_S = 6368/3 * 0.21
delta_t = 0.3/365.25   # uncertainty window in years

probs = []
    
xrb_fracs = np.linspace(0, 1, 101)

for frac in tqdm(xrb_fracs, desc="Calculating probabilities"):
    
    grb_prob = brute_force_coincidence_fraction(lambda_G*frac, lambda_S, delta_t, num_simulations=10000)
    probs.append(grb_prob)


Calculating probabilities:   0%|          | 0/101 [00:00<?, ?it/s]

Calculating probabilities: 100%|██████████| 101/101 [02:20<00:00,  1.39s/it]


In [4]:
probby = np.trapz(probs, xrb_fracs)
confidence_level = 1- probby # * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for brute is {probby*100:.2f}%, with a confidence level of: {z_score:.1f} sigma")

confidence_level = 1- probby * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for brute total is {probby*100:.2f}%, with a confidence level of: {z_score:.1f} sigma")
# print(sProb)

The z-score for brute is 15.34%, with a confidence level of: 1.4 sigma
The z-score for brute total is 15.34%, with a confidence level of: 3.3 sigma
